# Training LSTM Model - Dynamic Configuration

## Mục tiêu
Notebook training LSTM động có thể chạy cho bất kỳ configuration nào trong EXPERIMENTS.
Chỉ cần thay đổi CONFIG_NAME ở cell đầu tiên để train configuration khác.

## Cách sử dụng
1. Chọn CONFIG_NAME từ danh sách: '7n_1n', '30n_1n', '30n_7n', '30n_30n', '90n_7n', '90n_30n'
2. Run all cells
3. Kết quả sẽ được lưu tự động theo configuration

## Troubleshooting
- **LSTM fails**: Install TensorFlow với `pip install tensorflow`
- **Memory issues**: Giảm batch_size trong LSTM_PARAMS
- **Long training time**: Giảm epochs hoặc patience trong LSTM_PARAMS
- **Overfitting**: Tăng dropout hoặc patience

In [ ]:
# ===============================================
# CONFIGURATION - THAY ĐỔI TẠI ĐÂY
# ===============================================
CONFIG_NAME = '90n_30n'  # Thay đổi theo experiment muốn chạy

# Available configurations:
# '7n_1n'    : 7 days → 1 day
# '30n_1n'   : 30 days → 1 day  
# '30n_7n'   : 30 days → 7 days
# '30n_30n'  : 30 days → 30 days
# '90n_7n'   : 90 days → 7 days
# '90n_30n'  : 90 days → 30 days

In [ ]:
import sys
sys.path.append('../src')
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json
import os
import time
import importlib
from datetime import datetime

# Enhanced TensorFlow availability check from notebook 05
TF_AVAILABLE = False
try:
    import tensorflow as tf
    TF_AVAILABLE = True
    print(f"✅ TensorFlow {tf.__version__} available")
    
    # Check for GPU
    if tf.config.list_physical_devices('GPU'):
        print(f"🚀 GPU acceleration available")
    else:
        print(f"💻 Using CPU for training")
        
    # Import LSTM trainer
    from lstm_trainer import train_lstm_model
    
    # Force reload config module to pick up changes
    import config
    importlib.reload(config)
    from config import EXPERIMENTS, LSTM_PARAMS, RANDOM_SEED
    
    # Reload the module to pick up any changes
    import lstm_trainer
    importlib.reload(lstm_trainer)
    from lstm_trainer import train_lstm_model
    
    print(f"✅ LSTM trainer imported successfully")
    
except ImportError as e:
    print(f"❌ TensorFlow not available: {e}")
    print(f"⚠️ LSTM training will fail")
    print("💡 Please install TensorFlow: pip install tensorflow")
    raise ImportError("TensorFlow required for LSTM training")

warnings.filterwarnings('ignore')
np.random.seed(RANDOM_SEED)
if TF_AVAILABLE:
    tf.random.set_seed(RANDOM_SEED)

plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("husl")

# Validate configuration
if CONFIG_NAME not in EXPERIMENTS:
    raise ValueError(f"Invalid CONFIG_NAME: {CONFIG_NAME}. Available: {list(EXPERIMENTS.keys())}")

# Get experiment configuration
experiment_config = EXPERIMENTS[CONFIG_NAME]
N_DAYS = experiment_config['N']
M_DAYS = experiment_config['M']
DESCRIPTION = experiment_config['description']

print(f"✅ Configuration loaded: {CONFIG_NAME}")
print(f"📊 Experiment: {DESCRIPTION}")
print(f"📥 Input: {N_DAYS} days")
print(f"📤 Output: {M_DAYS} days")
print(f"🎲 Random seed: {RANDOM_SEED}")
print(f"⏰ Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔄 Config and LSTM trainer modules reloaded")

## Configuration-Specific Information

In [ ]:
MODEL_TYPE = 'LSTM'

print(f"=== TRAINING {MODEL_TYPE} MODEL: {CONFIG_NAME} ===")
print(f"Cấu hình: {N_DAYS} ngày sequence → {M_DAYS} ngày dự đoán")
print(f"Mô tả: {DESCRIPTION}")
print(f"Target: Mực nước Cần Thơ")
print(f"Model: {MODEL_TYPE} với sequence input")

# Expected data characteristics
features_per_interval = 6  # 3 stations * 2 parameters
timesteps = N_DAYS

print(f"\nExpected data characteristics:")
print(f"  Input shape: (samples, {timesteps}, {features_per_interval})")
print(f"  - Timesteps: {timesteps} ( {8 * N_DAYS} points)")
print(f"  - Features: {features_per_interval} (3 stations × 2 parameters)")
print(f"  Output type: {'Single value' if M_DAYS == 1 else f'Aggregated over {M_DAYS} days'}")

print(f"\nGrid search parameters:")
for param, values in LSTM_PARAMS.items():
    if isinstance(values, list):
        print(f"  {param}: {values}")
    else:
        print(f"  {param}: {values}")
        
param_combinations = np.prod([len(v) for v in LSTM_PARAMS.values() if isinstance(v, list)])
print(f"\nTotal hyperparameter combinations: {param_combinations}")
print(f"Max epochs per combination: {LSTM_PARAMS['epochs']}")
print(f"Early stopping patience: {LSTM_PARAMS['patience']}")

# Warning about LSTM challenges based on notebook 05 analysis
print(f"\n⚠️  LSTM Training Considerations:")
print(f"   • LSTM models are prone to overfitting on small datasets")
print(f"   • Current config may need tuning (epochs={LSTM_PARAMS['epochs']}, patience={LSTM_PARAMS['patience']})")
print(f"   • Consider increasing validation_split if performance is poor")
print(f"   • Monitor train vs validation loss closely")
print(f"   • Expected training time: {param_combinations * 2:.0f}-{param_combinations * 5:.0f} minutes")

## Kiểm tra dữ liệu

In [ ]:
# Enhanced data validation from notebook 05
data_folder = '../data'
config_folder = f"{data_folder}/{CONFIG_NAME}_lstm"

print(f"🔍 Checking LSTM data availability for {CONFIG_NAME}...")

def validate_lstm_data_for_config():
    """Validate LSTM data availability for the selected configuration"""
    if not os.path.exists(config_folder):
        return False, f"LSTM data folder does not exist: {config_folder}"
    
    # Check required files
    required_files = ['X_train.npy', 'X_test.npy', 'y_train.npy', 'y_test.npy', 'metadata.json']
    missing_files = []
    
    for file in required_files:
        file_path = f"{config_folder}/{file}"
        if not os.path.exists(file_path):
            missing_files.append(file)
    
    if missing_files:
        return False, f"Missing LSTM files: {missing_files}"
    
    return True, "All LSTM data files available"

# Validate data
data_ready, message = validate_lstm_data_for_config()

if data_ready:
    print(f"✅ {message}")
    
    # Load metadata
    metadata_file = f"{config_folder}/metadata.json"
    with open(metadata_file, 'r') as f:
        metadata = json.load(f)
    
    print(f"\n📊 LSTM Data Information:")
    print(f"  X_train shape: {metadata['X_train_shape']}")
    print(f"  X_test shape: {metadata['X_test_shape']}")
    print(f"  y_train shape: {metadata['y_train_shape']}")
    print(f"  y_test shape: {metadata['y_test_shape']}")
    print(f"  Target: {metadata['target_col']}")
    print(f"  Created: {metadata.get('created_at', 'Unknown')}")
    
    # Validate expected vs actual shape
    expected_shape = (None, timesteps, features_per_interval)  # None for variable batch size
    actual_train_shape = metadata['X_train_shape']
    
    if len(actual_train_shape) == 3 and actual_train_shape[1:] == expected_shape[1:]:
        print(f"  ✅ Data shape matches expectation: {actual_train_shape}")
    else:
        print(f"  ⚠️  Data shape mismatch: expected (?, {timesteps}, {features_per_interval}), got {actual_train_shape}")
        
    # Check data size for overfitting warning
    n_samples = actual_train_shape[0]
    if n_samples < 1000:
        print(f"  ⚠️  Small dataset ({n_samples} samples) - high overfitting risk!")
    elif n_samples < 2000:
        print(f"  ⚠️  Medium dataset ({n_samples} samples) - monitor overfitting")
    else:
        print(f"  ✅ Good dataset size ({n_samples} samples)")
        
else:
    print(f"❌ {message}")
    print(f"💡 Please run notebook 02_feature_engineering.ipynb first")
    raise RuntimeError(f"LSTM data not ready for {CONFIG_NAME}")

# Check if model already exists
model_folder = f"../models/{CONFIG_NAME}_lstm"
model_file = f"{model_folder}/best_model.keras"

if os.path.exists(model_file):
    print(f"\n⚠️  LSTM model already exists: {model_file}")
    print(f"🔄 Training will overwrite existing model.")
else:
    print(f"\n🆕 Training new LSTM model for {CONFIG_NAME}")

print(f"\n🎯 Ready to start LSTM training!")

## Training với Grid Search

In [ ]:
# Enhanced LSTM training with better error handling from notebook 05
# Extract epochs and patience from param_grid like in notebook 05
epochs = LSTM_PARAMS.get('epochs', [100])[0] if isinstance(LSTM_PARAMS.get('epochs', [100]), list) else LSTM_PARAMS.get('epochs', 100)
patience = LSTM_PARAMS.get('patience', [5])[0] if isinstance(LSTM_PARAMS.get('patience', [5]), list) else LSTM_PARAMS.get('patience', 5)

# Prepare parameter grid (remove epochs and patience)
lstm_param_grid = LSTM_PARAMS.copy()
lstm_param_grid.pop('epochs', None)
lstm_param_grid.pop('patience', None)

print(f"\n🚀 Starting LSTM training for {CONFIG_NAME}...")
print(f"📊 Experiment: {DESCRIPTION}")
print(f"🔢 Parameter combinations: {param_combinations}")
print(f"📈 Epochs per combination: {epochs}")
print(f"⏳ Early stopping patience: {patience}")
print(f"⏰ Expected training time: {param_combinations * 3:.0f}-{param_combinations * 8:.0f} minutes")
print(f"🎯 Parameters: {lstm_param_grid}")

# GPU optimization for small datasets
print(f"\n⚡ GPU Optimization for small dataset:")
n_samples = 914  # From data loading output
if n_samples < 2000:
    print(f"  📊 Small dataset detected ({n_samples} samples)")
    print(f"  🔧 Optimizing for fast training...")
    
    # Reduce epochs for small datasets (faster convergence expected)
    if epochs > 30:
        epochs = min(30, epochs)
        print(f"  📉 Reduced max epochs to {epochs} (small dataset converges faster)")
    
    # Increase patience slightly to avoid premature stopping
    if patience < 7:
        patience = 7
        print(f"  ⏱️ Increased patience to {patience} (avoid early stopping)")
    
    # Prefer larger batch sizes for GPU efficiency
    if 'batch_size' in lstm_param_grid:
        original_batches = lstm_param_grid['batch_size']
        lstm_param_grid['batch_size'] = [b for b in original_batches if b >= 32]
        if not lstm_param_grid['batch_size']:  # If all filtered out
            lstm_param_grid['batch_size'] = [32]
        print(f"  🚀 Optimized batch sizes: {lstm_param_grid['batch_size']} (GPU efficiency)")
    
    # Reduce model complexity for small datasets
    if 'units' in lstm_param_grid:
        original_units = lstm_param_grid['units']
        lstm_param_grid['units'] = [u for u in original_units if u <= 100]
        if not lstm_param_grid['units']:
            lstm_param_grid['units'] = [50, 100]
        print(f"  🧠 Reduced LSTM units: {lstm_param_grid['units']} (prevent overfitting)")

# Memory management for large sequences
timesteps = N_DAYS
if timesteps > 2000:  # Very long sequences
    print(f"\n⚠️  Long sequence detected ({timesteps} timesteps)")
    print(f"Consider reducing batch sizes if memory issues occur")
    # Reduce batch sizes for memory efficiency
    if 'batch_size' in lstm_param_grid:
        lstm_param_grid['batch_size'] = [b for b in lstm_param_grid['batch_size'] if b <= 32]
        print(f"Adjusted batch sizes: {lstm_param_grid['batch_size']}")

# Update param combinations after optimization
optimized_combinations = np.prod([len(v) for v in lstm_param_grid.values() if isinstance(v, list)])
print(f"\n🎯 Optimized training plan:")
print(f"  Original combinations: {param_combinations}")
print(f"  Optimized combinations: {optimized_combinations}")
print(f"  Max epochs: {epochs}")
print(f"  Expected time: {optimized_combinations * 1:.0f}-{optimized_combinations * 3:.0f} minutes")

start_time = time.time()

try:
    print(f"\n📊 Starting LSTM grid search...")
    print(f"⚠️  Note: LSTM models may overfit on small datasets")
    
    trainer = train_lstm_model(
        config_name=CONFIG_NAME,
        param_grid=lstm_param_grid,  # Pass grid without epochs/patience
        data_folder='../data',
        models_folder='../models',
        epochs=epochs,  # Pass as separate parameter
        patience=patience,  # Pass as separate parameter
        validation_split=0.2,
        verbose=0  # Reduce verbosity for faster training
    )
    
    training_time = time.time() - start_time
    print(f"\n✅ Training completed in {training_time/60:.1f} minutes")
    print(f"⚡ Speed: {training_time/optimized_combinations:.1f} seconds per combination")
    
except Exception as e:
    training_time = time.time() - start_time
    print(f"\n❌ Training failed after {training_time/60:.1f} minutes")
    print(f"Error: {e}")
    
    # Common LSTM training issues and suggestions
    if "memory" in str(e).lower() or "oom" in str(e).lower():
        print(f"\n💡 Memory issue detected for {CONFIG_NAME}:")
        print(f"  - Try reducing batch_size in config.py")
        print(f"  - Consider reducing LSTM units")
        print(f"  - Current sequence length: {timesteps} (very long for {N_DAYS} days)")
    elif "shape" in str(e).lower():
        print(f"\n💡 Shape mismatch detected:")
        print(f"  - Check feature engineering output for {CONFIG_NAME}")
        print(f"  - Verify data preprocessing steps")
    
    raise e

## Phân tích kết quả theo Configuration

In [ ]:
# Load và hiển thị kết quả chi tiết
results_folder = f"../models/{CONFIG_NAME}_lstm"

# Load results
with open(f"{results_folder}/results.json", 'r') as f:
    results = json.load(f)

print(f"=== KẾT QUẢ TRAINING {CONFIG_NAME} ===")
print(f"Configuration: {DESCRIPTION}")
print(f"Model type: {results['model_type']}")

print(f"\nBest hyperparameters:")
for param, value in results['best_params'].items():
    print(f"  {param}: {value}")

print(f"\nBest validation loss: {results['best_val_loss']:.6f}")
print(f"Training epochs: {results['training_epochs']}")

print(f"\nTraining metrics:")
for metric, value in results['train_metrics'].items():
    print(f"  {metric}: {value:.6f}")

print(f"\nTest metrics:")
for metric, value in results['test_metrics'].items():
    print(f"  {metric}: {value:.6f}")

print(f"\nModel architecture:")
print(f"  Total parameters: {results['model_params']['total_params']:,}")
print(f"  Trainable parameters: {results['model_params']['trainable_params']:,}")
print(f"  Grid search combinations: {results['grid_search_combinations']}")
print(f"  Data shapes: {results['data_shapes']}")
print(f"  Training samples: {results['data_shapes']['X_train'][0]:,}")
print(f"  Test samples: {results['data_shapes']['X_test'][0]:,}")

# Performance interpretation based on configuration
rmse = results['test_metrics']['RMSE']
r2 = results['test_metrics']['R2']
val_loss = results['best_val_loss']

print(f"\n📊 Performance Assessment cho {CONFIG_NAME}:")
if r2 > 0.95:
    performance = "Excellent (R² > 0.95)"
elif r2 > 0.90:
    performance = "Very Good (R² > 0.90)"
elif r2 > 0.80:
    performance = "Good (R² > 0.80)"
elif r2 > 0.70:
    performance = "Fair (R² > 0.70)"
else:
    performance = "Needs Improvement (R² ≤ 0.70)"

print(f"  Overall: {performance}")
print(f"  RMSE: {rmse:.4f} m (±{rmse*100:.1f} cm average error)")
print(f"  R²: {r2:.4f} ({r2*100:.1f}% variance explained)")
print(f"  Validation Loss: {val_loss:.6f}")

# Configuration-specific insights
if M_DAYS == 1:
    print(f"  Short-term prediction (1 day): {'Good for LSTM' if r2 > 0.80 else 'May need more data or tuning'}")
elif M_DAYS <= 7:
    print(f"  Medium-term prediction ({M_DAYS} days): {'LSTM handles well' if r2 > 0.70 else 'Challenging for sequential model'}")
else:
    print(f"  Long-term prediction ({M_DAYS} days): {'Impressive for LSTM' if r2 > 0.60 else 'Expected difficulty for long-term'}")

timesteps = N_DAYS
if timesteps >= 2000:
    print(f"  Very long sequences ({timesteps}): {'LSTM captures long patterns well' if r2 > 0.75 else 'May suffer from vanishing gradients'}")
elif timesteps >= 500:
    print(f"  Long sequences ({timesteps}): {'Good for temporal patterns' if r2 > 0.80 else 'Consider shorter sequences'}")
else:
    print(f"  Short sequences ({timesteps}): {'Efficient processing' if r2 > 0.85 else 'May need longer history'}")

# Training efficiency analysis
avg_epochs = results['training_epochs']
# Handle LSTM_PARAMS['epochs'] which might be a list
max_epochs_param = LSTM_PARAMS['epochs']
max_epochs = max_epochs_param[0] if isinstance(max_epochs_param, list) else max_epochs_param
if avg_epochs < max_epochs * 0.3:
    print(f"  Training efficiency: Early convergence ({avg_epochs}/{max_epochs} epochs)")
elif avg_epochs >= max_epochs * 0.8:
    print(f"  Training efficiency: May need more epochs ({avg_epochs}/{max_epochs})")
else:
    print(f"  Training efficiency: Good convergence ({avg_epochs}/{max_epochs} epochs)")

## Grid Search Analysis

In [ ]:
# Load grid search results
grid_results = pd.read_csv(f"{results_folder}/grid_search_results_full.csv")

print(f"=== GRID SEARCH RESULTS - {CONFIG_NAME} ===")
print(f"Total combinations tested: {len(grid_results)}")
print(f"Configuration: {DESCRIPTION}")

# Remove failed experiments
successful_results = grid_results[grid_results['best_val_loss'] != float('inf')].copy()
failed_results = grid_results[grid_results['best_val_loss'] == float('inf')]

print(f"Successful combinations: {len(successful_results)}")
if len(failed_results) > 0:
    print(f"Failed combinations: {len(failed_results)}")
    print(f"Common failure reasons: {failed_results['error'].value_counts().head(3).to_dict() if 'error' in failed_results.columns else 'Not recorded'}")

if len(successful_results) > 0:
    # Top 10 best combinations
    top_combinations = successful_results.nsmallest(10, 'best_val_loss')[[
        'units', 'n_layers', 'dropout', 'batch_size', 'best_val_loss', 'epochs_trained'
    ]]
    
    print(f"\nTop 10 combinations for {CONFIG_NAME} (by validation loss):")
    print(top_combinations.to_string(index=False))
    
    # Parameter analysis with configuration context
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'Hyperparameter Effects - {CONFIG_NAME} LSTM ({DESCRIPTION})', fontsize=16)
    
    # Units effect
    units_effect = successful_results.groupby('units')['best_val_loss'].agg(['mean', 'std', 'count'])
    axes[0,0].errorbar(units_effect.index, units_effect['mean'], yerr=units_effect['std'], 
                      marker='o', capsize=5)
    axes[0,0].set_xlabel('LSTM Units')
    axes[0,0].set_ylabel('Validation Loss')
    axes[0,0].set_title('Effect of LSTM Units')
    axes[0,0].grid(True, alpha=0.3)
    
    # Layers effect
    layers_effect = successful_results.groupby('n_layers')['best_val_loss'].agg(['mean', 'std', 'count'])
    axes[0,1].errorbar(layers_effect.index, layers_effect['mean'], yerr=layers_effect['std'], 
                      marker='o', capsize=5)
    axes[0,1].set_xlabel('Number of LSTM Layers')
    axes[0,1].set_ylabel('Validation Loss')
    axes[0,1].set_title('Effect of Number of Layers')
    axes[0,1].grid(True, alpha=0.3)
    
    # Dropout effect
    dropout_effect = successful_results.groupby('dropout')['best_val_loss'].agg(['mean', 'std', 'count'])
    axes[1,0].errorbar(dropout_effect.index, dropout_effect['mean'], yerr=dropout_effect['std'], 
                      marker='o', capsize=5)
    axes[1,0].set_xlabel('Dropout Rate')
    axes[1,0].set_ylabel('Validation Loss')
    axes[1,0].set_title('Effect of Dropout')
    axes[1,0].grid(True, alpha=0.3)
    
    # Batch size effect
    batch_effect = successful_results.groupby('batch_size')['best_val_loss'].agg(['mean', 'std', 'count'])
    axes[1,1].errorbar(batch_effect.index, batch_effect['mean'], yerr=batch_effect['std'], 
                      marker='o', capsize=5)
    axes[1,1].set_xlabel('Batch Size')
    axes[1,1].set_ylabel('Validation Loss')
    axes[1,1].set_title('Effect of Batch Size')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Configuration-specific hyperparameter insights
    best_params = results['best_params']
    print(f"\n🔧 Hyperparameter insights for {CONFIG_NAME}:")
    
    if best_params['units'] >= 120:
        print(f"  - High LSTM units ({best_params['units']}) needed for complex {N_DAYS}→{M_DAYS} patterns")
    elif best_params['units'] <= 60:
        print(f"  - Low LSTM units ({best_params['units']}) sufficient, good efficiency")
    
    if best_params['n_layers'] >= 2:
        print(f"  - Deep LSTM ({best_params['n_layers']} layers) captures hierarchical patterns")
    else:
        print(f"  - Single LSTM layer sufficient for {CONFIG_NAME}")
    
    if best_params['dropout'] >= 0.25:
        print(f"  - High dropout ({best_params['dropout']}) prevents overfitting")
    elif best_params['dropout'] <= 0.1:
        print(f"  - Low dropout ({best_params['dropout']}), model generalizes well")
    
    if best_params['batch_size'] <= 32:
        print(f"  - Small batch size ({best_params['batch_size']}) for stable training")
    elif best_params['batch_size'] >= 64:
        print(f"  - Large batch size ({best_params['batch_size']}) for efficient training")
        
else:
    print("\n❌ No successful grid search results found!")
    print("All hyperparameter combinations failed. Check:")
    print("- Data format and shapes")
    print("- Memory availability")
    print("- TensorFlow installation")

## Training History Analysis

In [ ]:
# Load training history
if os.path.exists(f"{results_folder}/training_history.csv"):
    training_history = pd.read_csv(f"{results_folder}/training_history.csv")
    
    print(f"=== TRAINING HISTORY - {CONFIG_NAME} ===")
    print(f"Configuration: {DESCRIPTION}")
    print(f"Total epochs: {len(training_history)}")
    print(f"Final training loss: {training_history['loss'].iloc[-1]:.6f}")
    print(f"Final validation loss: {training_history['val_loss'].iloc[-1]:.6f}")
    print(f"Best validation loss: {training_history['val_loss'].min():.6f}")
    print(f"Best epoch: {training_history['val_loss'].idxmin() + 1}")
    
    # Check for overfitting
    final_train_loss = training_history['loss'].iloc[-1]
    final_val_loss = training_history['val_loss'].iloc[-1]
    overfitting_ratio = final_val_loss / final_train_loss
    
    if overfitting_ratio > 1.5:
        print(f"⚠️  Potential overfitting detected (val/train loss ratio: {overfitting_ratio:.2f})")
    elif overfitting_ratio < 1.1:
        print(f"✅ Good generalization (val/train loss ratio: {overfitting_ratio:.2f})")
    else:
        print(f"✅ Acceptable generalization (val/train loss ratio: {overfitting_ratio:.2f})")
    
    # Plot training history with configuration-specific title
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle(f'Training History - {CONFIG_NAME} ({DESCRIPTION})', fontsize=16)
    
    # Loss curves
    axes[0].plot(training_history.index + 1, training_history['loss'], 'b-', label='Training Loss')
    axes[0].plot(training_history.index + 1, training_history['val_loss'], 'r-', label='Validation Loss')
    axes[0].axvline(training_history['val_loss'].idxmin() + 1, color='green', linestyle='--', 
                   label=f'Best Epoch ({training_history["val_loss"].idxmin() + 1})')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss (MSE)')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # MAE curves
    if 'mae' in training_history.columns:
        axes[1].plot(training_history.index + 1, training_history['mae'], 'b-', label='Training MAE')
        axes[1].plot(training_history.index + 1, training_history['val_mae'], 'r-', label='Validation MAE')
        axes[1].axvline(training_history['val_loss'].idxmin() + 1, color='green', linestyle='--',
                       label=f'Best Epoch ({training_history["val_loss"].idxmin() + 1})')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('MAE')
        axes[1].set_title('Training and Validation MAE')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Training insights based on configuration
    print(f"\n📈 Training insights for {CONFIG_NAME}:")
    
    convergence_epoch = training_history['val_loss'].idxmin() + 1
    total_epochs = len(training_history)
    
    if convergence_epoch < total_epochs * 0.3:
        print(f"  - Fast convergence (epoch {convergence_epoch}/{total_epochs})")
        print(f"  - Model learned patterns quickly for {N_DAYS}→{M_DAYS}")
    elif convergence_epoch > total_epochs * 0.8:
        print(f"  - Slow convergence (epoch {convergence_epoch}/{total_epochs})")
        print(f"  - Complex {N_DAYS}→{M_DAYS} pattern requires more training")
    else:
        print(f"  - Normal convergence (epoch {convergence_epoch}/{total_epochs})")
        
    # Learning curve analysis
    early_loss = training_history['val_loss'].iloc[:5].mean()
    final_loss = training_history['val_loss'].iloc[-5:].mean()
    improvement = (early_loss - final_loss) / early_loss * 100
    
    print(f"  - Validation loss improvement: {improvement:.1f}%")
    if improvement > 50:
        print(f"  - Excellent learning progress")
    elif improvement > 20:
        print(f"  - Good learning progress")
    else:
        print(f"  - Limited improvement, may need different architecture")
        
else:
    print(f"⚠️  Training history not found: {results_folder}/training_history.csv")

## Model Predictions Visualization için Configuration

In [ ]:
# Load model và dự đoán
from tensorflow.keras.models import load_model
import tensorflow as tf

# Handle Keras serialization issues
try:
    # Try loading with compile=False to avoid metric issues
    model = load_model(f"{results_folder}/best_model.h5", compile=False)
    
    # Recompile the model with standard metrics
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    print("✅ Model loaded successfully (recompiled)")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    # If loading fails, try with custom objects
    try:
        custom_objects = {
            'mse': tf.keras.metrics.MeanSquaredError(),
            'mae': tf.keras.metrics.MeanAbsoluteError()
        }
        model = load_model(f"{results_folder}/best_model.h5", custom_objects=custom_objects)
        print("✅ Model loaded with custom objects")
    except Exception as e2:
        print(f"❌ Failed to load model: {e2}")
        raise e2

print(f"=== MODEL ARCHITECTURE - {CONFIG_NAME} ===")
print(f"Configuration: {DESCRIPTION}")
model.summary()

# Load test data
X_test = np.load(f"../data/{CONFIG_NAME}_lstm/X_test.npy")
y_test = np.load(f"../data/{CONFIG_NAME}_lstm/y_test.npy")
datetime_test = pd.read_csv(f"../data/{CONFIG_NAME}_lstm/datetime_test.csv")
datetime_test['datetime'] = datetime_test['0']
datetime_test['datetime'] = pd.to_datetime(datetime_test['datetime'])

# Handle y_test shape based on configuration
if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    print(f"Multi-step target shape: {y_test.shape}")
    if M_DAYS > 1:
        print(f"Using mean of {M_DAYS}-day prediction period")
    y_test = np.mean(y_test, axis=1)
elif len(y_test.shape) > 1:
    y_test = y_test.squeeze()

print(f"\nTest data shapes for {CONFIG_NAME}:")
print(f"X_test: {X_test.shape} (samples, timesteps, features)")
print(f"y_test: {y_test.shape}")
print(f"Input sequence length: {X_test.shape[1]} timesteps ({X_test.shape[1]/96:.1f} days)")
print(f"Features per timestep: {X_test.shape[2]}")

# Predictions
print(f"\nGenerating predictions for {len(X_test)} samples...")
y_pred = model.predict(X_test, verbose=0).squeeze()

print(f"Predictions shape: {y_pred.shape}")
print(f"Prediction range: [{y_pred.min():.3f}, {y_pred.max():.3f}]")
print(f"Actual range: [{y_test.min():.3f}, {y_test.max():.3f}]")

In [ ]:
# Plot predictions vs actual with configuration-specific analysis
fig, axes = plt.subplots(2, 1, figsize=(16, 12))
fig.suptitle(f'LSTM Performance - {CONFIG_NAME} ({DESCRIPTION})', fontsize=16)

# Time series plot (first 1000 points)
n_points = min(1000, len(y_test))
axes[0].plot(datetime_test['datetime'][:n_points], y_test[:n_points], 'b-', label='Actual', alpha=0.7, linewidth=1)
axes[0].plot(datetime_test['datetime'][:n_points], y_pred[:n_points], 'r-', label='Predicted', alpha=0.7, linewidth=1)
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Water Level (m)')
axes[0].set_title(f'Time Series: Actual vs Predicted (First {n_points} points)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Add performance metrics to time series plot
axes[0].text(0.02, 0.98, f'RMSE: {rmse:.4f}m\nR²: {r2:.4f}\nInput: {N_DAYS}d→{M_DAYS}d', 
            transform=axes[0].transAxes, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Scatter plot
axes[1].scatter(y_test, y_pred, alpha=0.5, s=1)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual Water Level (m)')
axes[1].set_ylabel('Predicted Water Level (m)')
axes[1].set_title(f'Scatter Plot: Actual vs Predicted')
axes[1].grid(True, alpha=0.3)

# Add R² annotation with configuration info
axes[1].text(0.05, 0.95, f'R² = {r2:.4f}\nRMSE = {rmse:.4f}m\n{N_DAYS}d → {M_DAYS}d', 
            transform=axes[1].transAxes, 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# Residuals analysis
residuals = y_test - y_pred

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle(f'Residuals Analysis - {CONFIG_NAME} LSTM', fontsize=16)

# Residuals histogram
axes[0].hist(residuals, bins=50, alpha=0.7, edgecolor='black')
axes[0].axvline(residuals.mean(), color='red', linestyle='--', label=f'Mean: {residuals.mean():.4f}')
axes[0].set_xlabel('Residuals (m)')
axes[0].set_ylabel('Frequency')
axes[0].set_title(f'Residuals Distribution')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals vs predicted
axes[1].scatter(y_pred, residuals, alpha=0.5, s=1)
axes[1].axhline(y=0, color='red', linestyle='--')
axes[1].set_xlabel('Predicted Values (m)')
axes[1].set_ylabel('Residuals (m)')
axes[1].set_title('Residuals vs Predicted Values')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n=== RESIDUALS ANALYSIS - {CONFIG_NAME} ===")
print(f"Configuration: {DESCRIPTION}")
print(f"Mean residual: {residuals.mean():.6f} m")
print(f"Std residual: {residuals.std():.6f} m")
print(f"Min residual: {residuals.min():.6f} m")
print(f"Max residual: {residuals.max():.6f} m")
print(f"95% of errors within: ±{np.percentile(np.abs(residuals), 95):.4f} m")

# Configuration-specific residual insights
if abs(residuals.mean()) < 0.001:
    print(f"✅ Unbiased LSTM predictions (mean residual ≈ 0)")
else:
    bias_direction = "over-predicting" if residuals.mean() < 0 else "under-predicting"
    print(f"⚠️ LSTM bias detected: {bias_direction} by {abs(residuals.mean()):.4f}m on average")

# Compare with configuration complexity
if timesteps > 1000 and r2 > 0.85:
    print(f"✅ LSTM handles long sequences ({timesteps}) very well")
elif timesteps > 1000 and r2 < 0.75:
    print(f"⚠️ LSTM struggles with very long sequences ({timesteps})")
    print(f"  Consider reducing sequence length or using attention mechanisms")

## Sequence Analysis

In [ ]:
# Phân tích một vài sequences cụ thể cho configuration
sample_indices = [0, len(X_test)//4, len(X_test)//2, len(X_test)*3//4]
feature_names = ['Can Tho Rainfall', 'Can Tho Water Level', 'Chau Doc Rainfall', 
                'Chau Doc Water Level', 'Dai Ngai Rainfall', 'Dai Ngai Water Level']

fig, axes = plt.subplots(2, 2, figsize=(20, 12))
axes = axes.flatten()
fig.suptitle(f'Input Sequences Analysis - {CONFIG_NAME} ({DESCRIPTION})', fontsize=16)

for i, idx in enumerate(sample_indices):
    if idx < len(X_test):
        # Plot water level features only (more relevant)
        for feat_idx, feat_name in enumerate(feature_names):
            if 'Water Level' in feat_name:
                # Sample every 10th point for long sequences to avoid overcrowding
                if timesteps > 1000:
                    step = max(1, timesteps // 100)
                    x_seq = X_test[idx, ::step, feat_idx]
                    x_axis = np.arange(0, timesteps, step)
                else:
                    x_seq = X_test[idx, :, feat_idx]
                    x_axis = np.arange(timesteps)
                    
                axes[i].plot(x_axis, x_seq, label=feat_name, linewidth=1.5)
        
        axes[i].axhline(y=y_test[idx], color='blue', linestyle='--', linewidth=2, 
                       label=f'Actual: {y_test[idx]:.3f}')
        axes[i].axhline(y=y_pred[idx], color='red', linestyle='--', linewidth=2, 
                       label=f'Predicted: {y_pred[idx]:.3f}')
        
        error = abs(y_test[idx] - y_pred[idx])
        axes[i].set_title(f'Sample {idx}: Error = {error:.4f}m\n({N_DAYS}d input → {M_DAYS}d prediction)')
        axes[i].set_xlabel(f'Timestep (15-min intervals, {timesteps} total)')
        axes[i].set_ylabel('Normalized Water Level')
        axes[i].legend(fontsize=8)
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Sequence complexity analysis
print(f"\n🔍 Sequence Analysis for {CONFIG_NAME}:")
print(f"Input sequence length: {timesteps} timesteps ({N_DAYS} days)")
print(f"Features per timestep: {features_per_interval}")
print(f"Total input data points: {timesteps * features_per_interval:,}")

if timesteps > 2000:
    print(f"⚠️  Very long sequences may cause:")
    print(f"  - Memory issues during training")
    print(f"  - Vanishing gradient problems")
    print(f"  - Slower training and inference")
elif timesteps > 500:
    print(f"✅ Long sequences good for:")
    print(f"  - Capturing seasonal patterns")
    print(f"  - Long-term dependencies")
else:
    print(f"✅ Short sequences efficient for:")
    print(f"  - Fast training and inference")
    print(f"  - Recent pattern focus")

# Analyze prediction difficulty by configuration
prediction_difficulty = {
    1: "Easy (short-term)",
    7: "Medium (weekly)", 
    30: "Hard (monthly)"
}

difficulty = prediction_difficulty.get(M_DAYS, f"Very Hard ({M_DAYS} days)")
print(f"\nPrediction difficulty: {difficulty}")
print(f"LSTM performance: {'Excellent' if r2 > 0.9 else 'Good' if r2 > 0.8 else 'Acceptable' if r2 > 0.7 else 'Needs improvement'}")

## Kết luận

In [ ]:
print(f"\n" + "="*60)
print(f"KẾT LUẬN - {CONFIG_NAME} LSTM MODEL")
print("="*60)

print(f"\n🎯 Configuration: {DESCRIPTION}")
print(f"🧠 Model Type: {MODEL_TYPE} (Recurrent Neural Network)")
print(f"⏱️ Training Time: {training_time/60:.1f} minutes")
print(f"🎲 Random Seed: {RANDOM_SEED} (reproducible)")

print(f"\n📈 Performance Metrics:")
print(f"  Test RMSE: {results['test_metrics']['RMSE']:.6f} m (±{results['test_metrics']['RMSE']*100:.2f} cm)")
print(f"  Test MAE:  {results['test_metrics']['MAE']:.6f} m (±{results['test_metrics']['MAE']*100:.2f} cm)")
print(f"  Test R²:   {results['test_metrics']['R2']:.6f} ({results['test_metrics']['R2']*100:.2f}% variance explained)")
print(f"  Validation Loss: {results['best_val_loss']:.6f}")
print(f"  Assessment: {performance}")

print(f"\n🏗️ Architecture:")
print(f"  LSTM Units: {results['best_params']['units']}")
print(f"  LSTM Layers: {results['best_params']['n_layers']}")
print(f"  Dropout Rate: {results['best_params']['dropout']}")
print(f"  Batch Size: {results['best_params']['batch_size']}")
print(f"  Total Parameters: {results['model_params']['total_params']:,}")
print(f"  Training Epochs: {results['training_epochs']}/{LSTM_PARAMS['epochs']}")

print(f"\n🎯 Sequence Characteristics:")
print(f"  Input Shape: ({results['data_shapes']['X_test'][0]:,}, {timesteps}, {features_per_interval})")
print(f"  Sequence Length: {timesteps} timesteps ({N_DAYS} days)")
print(f"  Prediction Horizon: {M_DAYS} day{'s' if M_DAYS > 1 else ''}")
print(f"  Training Samples: {results['data_shapes']['X_train'][0]:,}")
print(f"  Test Samples: {results['data_shapes']['X_test'][0]:,}")

print(f"\n🔍 Key Insights for {CONFIG_NAME}:")

# Temporal pattern insights
if N_DAYS >= 365:
    print(f"  ✅ Captures full seasonal cycles ({N_DAYS} days input)")
elif N_DAYS >= 30:
    print(f"  ✅ Captures monthly patterns ({N_DAYS} days input)")
else:
    print(f"  ✅ Focuses on recent patterns ({N_DAYS} days input)")

# Prediction horizon insights
if M_DAYS == 1:
    print(f"  🎯 Short-term prediction: {'Excellent LSTM performance' if r2 > 0.9 else 'Good LSTM performance' if r2 > 0.8 else 'Acceptable performance'}")
elif M_DAYS <= 7:
    print(f"  🎯 Medium-term prediction: {'Impressive LSTM capability' if r2 > 0.8 else 'Reasonable LSTM performance' if r2 > 0.7 else 'Challenging prediction'}")
else:
    print(f"  🎯 Long-term prediction: {'Outstanding LSTM performance' if r2 > 0.7 else 'Acceptable given difficulty' if r2 > 0.6 else 'Difficult prediction task'}")

# Architecture insights
if results['best_params']['n_layers'] > 1:
    print(f"  🏗️ Deep LSTM architecture captures hierarchical temporal patterns")
else:
    print(f"  🏗️ Single LSTM layer sufficient for this configuration")

if timesteps > 1500:
    print(f"  ⚠️ Very long sequences ({timesteps}) - consider computational efficiency")
elif timesteps > 500:
    print(f"  ✅ Long sequences ({timesteps}) good for capturing patterns")
else:
    print(f"  ✅ Efficient sequence length ({timesteps}) for fast processing")

print(f"\n📁 Saved Files:")
print(f"  Model: ../models/{CONFIG_NAME}_lstm/best_model.h5")
print(f"  Results: ../models/{CONFIG_NAME}_lstm/results.json")
print(f"  Grid Search: ../models/{CONFIG_NAME}_lstm/grid_search_results.csv")
print(f"  Training History: ../models/{CONFIG_NAME}_lstm/training_history.csv")

print(f"\n🚀 Production Readiness:")
if r2 > 0.85 and abs(residuals.mean()) < 0.01:
    print(f"  ✅ LSTM model ready for production deployment")
    print(f"  ✅ Excellent sequential pattern recognition")
    print(f"  ✅ Unbiased predictions with good accuracy")
elif r2 > 0.75:
    print(f"  ⚠️ LSTM model acceptable but monitor performance")
    print(f"  💡 Consider ensemble with XGBoost for robustness")
else:
    print(f"  ❌ LSTM model needs improvement before production")
    print(f"  🔧 Consider architecture changes or different sequence length")

print(f"\n💡 LSTM-Specific Advantages for {CONFIG_NAME}:")
print(f"  ✅ Captures temporal dependencies naturally")
print(f"  ✅ Handles variable-length patterns")
print(f"  ✅ Memory of past states for context")
print(f"  ✅ Good for non-linear temporal relationships")

print(f"\n🔄 Next Steps:")
print(f"  1. Compare with XGBoost model for {CONFIG_NAME}")
print(f"  2. Analyze which approach works better for {N_DAYS}→{M_DAYS}")
print(f"  3. Consider ensemble combining both approaches")
print(f"  4. Test on different seasonal periods")

print(f"\n✅ {CONFIG_NAME} LSTM training completed successfully!")
print(f"🧠 Sequential model ready for time series prediction")